## Prerequisites
### Package Installation
```
$ sudo dnf install python2-devel python3-devel libcouchbase-devel libcouchbase-tools
$ pipenv install -d
```
If you get some errors during the `pipenv install` there's probably an issue with some C library somewhere. Have fun sorting that out!
### Couchbase Configuration
Follow the steps in presentation.md to configure the couchbase bucket, load the data into it, and create the spatial view.
### Enable the ArcGIS jupyter notebook widgets
Run these commands before starting the jupyter notebook server for the first time - the notebook server should be **stopped**.
```
$ jupyter nbextension install --py --sys-prefix arcgis
$ jupyter nbextension enable --py --sys-prefix arcgis
```

In [1]:
from arcgis.gis import GIS

In [2]:
gis = GIS()

In [3]:
map = gis.map("San Antonio, TX 78229")
map.basemap = 'osm'
map.zoom = 14

In [4]:
map

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

In [5]:
map.extent

{'type': 'extent',
 'xmin': -10976498.053468455,
 'ymin': 3437306.0202543377,
 'xmax': -10957866.52782397,
 'ymax': 3444949.7230828446,
 'spatialReference': {'wkid': 102100}}

In [6]:
xmin = map.extent['xmin']
ymin = map.extent['ymin']
xmax = map.extent['xmax']
ymax = map.extent['ymax']

In [7]:
import pyproj

In [8]:
# For some reason this is the only way to initialize Web Mercator
srcProj = pyproj.Proj(init='epsg:3857')
# Can also be initialized like so:
# Proj(proj="longlat", ellps="WGS84", datum="WGS84")
dstProj = pyproj.Proj(init='epsg:4326')

In [9]:
lower_left = pyproj.transform(srcProj, dstProj, xmin, ymin)
upper_right = pyproj.transform(srcProj, dstProj, xmax, ymax)

In [10]:
lower_left

(-98.60355967538888, 29.483313548576277)

In [11]:
upper_right

(-98.43618983285982, 29.54306834103877)

In [12]:
from cbgeo.utils import connect
from cbgeo.query import query_bounding_box

In [13]:
bucket = connect()

In [14]:
view = query_bounding_box(bucket, lower_left, upper_right)